In [ ]:
import torch
import stable_baselines3
import sys

print("python version:", sys.version)
print("stable_baselines3 version:", stable_baselines3.__version__)
print("torch version:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("cuda version:", torch.version.cuda)
print("cudnn version:", torch.backends.cudnn.version())

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# set torch default device
torch.set_default_device(device)


# Equation of Motion 3D Quadcopter

In [ ]:
# reload sympy
from sympy import *

# Equations of motion 3D quadcopter from https://arxiv.org/pdf/2304.13460.pdf

# w1,w2,w3,w4 are the motor speeds normalized to [-1,1]
# u1,u2,u3,u4 are the motor commands normalized to [-1,1]

state = symbols('x y z v_x v_y v_z phi theta psi p q r w1 w2 w3 w4')
x,y,z,vx,vy,vz,phi,theta,psi,p,q,r,w1,w2,w3,w4 = state
control = symbols('u_1 u_2 u_3 u_4')
u1,u2,u3,u4 = control

# parameters
g = 9.81
Ixx = 0.000906
Iyy = 0.001242
Izz = 0.002054

k_x  = 1.07933887e-05
k_y  = 9.65250793e-06
k_z  = 2.7862899e-05
k_w  = 4.36301076e-08
k_h  = 0.0625501332
k_p  = 1.4119331e-09
k_pv = -0.00797101848
k_q  = 1.21601884e-09
k_qv = 0.0129263739
k_r1 = 2.57035545e-06
k_r2 = 4.10923364e-07
k_rr = 0.000812932607

tau = 0.06
w_min = 3000
w_max = 12000


# Rotation matrix 
Rx = Matrix([[1, 0, 0], [0, cos(phi), -sin(phi)], [0, sin(phi), cos(phi)]])
Ry = Matrix([[cos(theta), 0, sin(theta)], [0, 1, 0], [-sin(theta), 0, cos(theta)]])
Rz = Matrix([[cos(psi), -sin(psi), 0], [sin(psi), cos(psi), 0], [0, 0, 1]])
R = Rz*Ry*Rx

# Body velocity
vbx, vby, vbz = R.T@Matrix([vx,vy,vz])

# normalized motor speeds to rpm
W1 = (w1+1)/2*(w_max-w_min) + w_min
W2 = (w2+1)/2*(w_max-w_min) + w_min
W3 = (w3+1)/2*(w_max-w_min) + w_min
W4 = (w4+1)/2*(w_max-w_min) + w_min

# first order delay
d_w1 = (u1 - w1)/tau
d_w2 = (u2 - w2)/tau
d_w3 = (u3 - w3)/tau
d_w4 = (u4 - w4)/tau

# derivative of rpm: d/dt[((w+1)/2*(w_max-w_min) + w_min)]
d_W1 = d_w1/2*(w_max-w_min)
d_W2 = d_w2/2*(w_max-w_min)
d_W3 = d_w3/2*(w_max-w_min)
d_W4 = d_w4/2*(w_max-w_min)

# Thrust and Drag
T = -k_w*(W1**2 + W2**2 + W3**2 + W4**2) - k_h*(vbx**2+vby**2) - k_z*vbz*(W1+W2+W3+W4)
Dx = -k_x*vbx*(W1+W2+W3+W4)
Dy = -k_y*vby*(W1+W2+W3+W4)

# Moments
Mx = k_p*(W1**2-W2**2-W3**2+W4**2) + k_pv*vy
My = k_q*(W1**2+W2**2-W3**2-W4**2) + k_qv*vx
Mz = k_r1*(-W1+W2-W3+W4) + k_r2*(-d_W1+d_W2-d_W3+d_W4) - k_rr*r

# Dynamics
d_x = vx
d_y = vy
d_z = vz

d_vx, d_vy, d_vz = Matrix([0,0,g]) + R@Matrix([Dx, Dy,T])

d_phi   = p + q*sin(phi)*tan(theta) + r*cos(phi)*tan(theta)
d_theta = q*cos(phi) - r*sin(phi)
d_psi   = q*sin(phi)/cos(theta) + r*cos(phi)/cos(theta)

d_p     = (q*r*(Iyy-Izz) + Mx)/Ixx
d_q     = (p*r*(Izz-Ixx) + My)/Iyy
d_r     = (p*q*(Ixx-Iyy) + Mz)/Izz

# State space model
f = [d_x, d_y, d_z, d_vx, d_vy, d_vz, d_phi, d_theta, d_psi, d_p, d_q, d_r, d_w1, d_w2, d_w3, d_w4]

# lambdify
f_func = lambdify((Array(state), Array(control)), Array(f), 'numpy')
# f_jax = lambdify((Array(state), Array(control)), Array(f), 'jax')
# f_num = lambdify((Array(state), Array(control)), Array(f), 'numexpr')
# f_cup = lambdify((Array(state), Array(control)), Array(f), 'cupy')


# Gym Environment

In [ ]:
# create a gym environment for the 3d quadcopter model
import gym
from gym import spaces
import numpy as np

class Quadcopter3D(gym.Env):
    def __init__(self):
        # Define the action space and observation space
        self.action_space = spaces.Box(low=-1, high=1, shape=(4,))
        # all states are  in [-inf, inf] except for the last 4 states which are in [w_min, w_max]
        self.observation_space = spaces.Box(low=np.array([-np.inf]*12+[-1]*4), high=np.array([-np.inf]*12+[1]*4), dtype=np.float32)
        
        # Define any other environment-specific parameters
        self.max_steps = 500  # Maximum number of steps in an episode
        self.dt = 0.01  # Time step duration
        self.goal_threshold = 0.1  # Threshold for considering the goal reached

        # Define any other necessary variables
        self.state = np.zeros(16)
        self.step_count = 0
        self.action = (0, 0, 0, 0)
    
    def reset(self):
        # Reset the state of the environment to an initial state
        x0 =  np.random.uniform(-5,5)
        y0 =  np.random.uniform(-5,5)
        z0 =  np.random.uniform(-5,5)

        vx0    = np.random.uniform(-1,1)
        vy0    = np.random.uniform(-1,1)
        vz0    = np.random.uniform(-1,1)

        phi0   = np.random.uniform(-1,1)
        theta0 = np.random.uniform(-1,1)
        psi0   = np.random.uniform(-np.pi,np.pi)

        p0     = np.random.uniform(-1,1)
        q0     = np.random.uniform(-1,1)
        r0     = np.random.uniform(-1,1)

        w10    = np.random.uniform(-1,1)
        w20    = np.random.uniform(-1,1)
        w30    = np.random.uniform(-1,1)
        w40    = np.random.uniform(-1,1)

        self.state = np.array([x0, y0, z0, vx0, vy0, vz0, phi0, theta0, psi0, p0, q0, r0, w10, w20, w30, w40])
        self.step_count = 0
        return self.state
    
    def step(self, action):
        # Simulate the environment one step forward in time with the given action
        self.step_count += 1        
        self.action = action
        self.state = self.state + self.dt*f_func(self.state, self.action)

        # Compute the reward
        x,y,z,vx,vy,vz,phi,theta,psi,p,q,r,w1,w2,w3,w4 = self.state
        #reward = 1-(x**2+y**2+z**2)/300

        # Rewards from https://github.com/uzh-rpg/flightmare/blob/competition/flightpy/configs/control/config.yaml
        pos_coeff = -0.02
        vel_coeff = -0.001
        ang_coeff = -0.02
        ang_vel_coeff = -0.001

        pos_reward = pos_coeff * np.linalg.norm(self.state[0:3])
        vel_reward = vel_coeff * np.linalg.norm(self.state[3:6])
        ang_reward = ang_coeff * np.linalg.norm(self.state[6:9])
        ang_vel_reward = ang_vel_coeff * np.linalg.norm(self.state[9:12])

        reward = 1+pos_reward + vel_reward + ang_reward + ang_vel_reward

        # Check if the goal has been reached
        goal_reached = np.linalg.norm(self.state[0:12]) < self.goal_threshold
        if goal_reached:
            reward = 100

        # Check if out of bounds
        out_of_bounds = np.any(np.abs(self.state[0:3]) > 10) or np.any(np.abs(self.state[6:9]) > np.pi)

        if out_of_bounds:
            reward = -1

        # Check if the episode is done
        done = goal_reached or out_of_bounds or self.step_count >= self.max_steps

        return self.state, reward, done, {}
    
    def render(self):
        # Outputs a dict containing all information for rendering
        state_dict = dict(zip(['x','y','z','vx','vy','vz','phi','theta','psi','p','q','r','w1','w2','w3','w4'], self.state))
        # Rescale actions to [0,1] for rendering
        action_dict = dict(zip(['u1','u2','u3','u4'], (np.array(self.action)+1)/2))
        return {**state_dict, **action_dict}

# Gym Vector Environment

In [ ]:
# Efficient vectorized version of the environment
import torch
from gym import spaces
from stable_baselines3.common.vec_env import VecEnv

class Quadcopter3DVec(VecEnv):
    def __init__(self, num_envs):
        # action space and observation space (with extra dimension for the number of environments)
        action_space = spaces.Box(low=-1, high=1, shape=(4,))
        observation_space = spaces.Box(low=np.array([-np.inf]*12+[-1]*4), high=np.array([-np.inf]*12+[1]*4))
        VecEnv.__init__(self,num_envs, observation_space, action_space)

        # Define any other environment-specific parameters
        self.max_steps = 1000  # Maximum number of steps in an episode
        self.dt = 0.01  # Time step duration

        # Define the hover state thresholds
        self.pos_threshold = 0.3  # max 30cm error
        self.vel_threshold = 0.3  # max 30cm/s error
        self.ang_threshold = 10*np.pi/180  # max 10 degree error
        self.rat_threshold = 10*np.pi/180  # max 10 degree/s error

        # Define any other necessary variables
        self.states = np.zeros((num_envs,16))
        self.step_counts = np.zeros(num_envs)
        self.actions = np.zeros((num_envs,4))

    def reset_(self, dones):
        num_reset = dones.sum()
        # Reset the state of the environment to an initial state
        x0 =  np.random.uniform(-5,5, size=(num_reset,))
        y0 =  np.random.uniform(-5,5, size=(num_reset,))
        z0 =  np.random.uniform(-5,5, size=(num_reset,))

        vx0    = np.random.uniform(-1,1, size=(num_reset,))
        vy0    = np.random.uniform(-1,1, size=(num_reset,))
        vz0    = np.random.uniform(-1,1, size=(num_reset,))

        phi0    = np.random.uniform(-1,1, size=(num_reset,))
        theta0  = np.random.uniform(-1,1, size=(num_reset,))
        psi0    = np.random.uniform(-np.pi, np.pi, size=(num_reset,))

        p0      = np.random.uniform(-1,1, size=(num_reset,))
        q0      = np.random.uniform(-1,1, size=(num_reset,))
        r0      = np.random.uniform(-1,1, size=(num_reset,))

        w10     = np.random.uniform(-1,1, size=(num_reset,))
        w20     = np.random.uniform(-1,1, size=(num_reset,))
        w30     = np.random.uniform(-1,1, size=(num_reset,))
        w40     = np.random.uniform(-1,1, size=(num_reset,))

        self.states[dones] = np.stack([x0, y0, z0, vx0, vy0, vz0, phi0, theta0, psi0, p0, q0, r0, w10, w20, w30, w40], axis=1)
        self.step_counts[dones] = np.zeros(num_reset)
        return self.states
    
    def reset(self):
        return self.reset_(np.ones(self.num_envs, dtype=bool))

    def step_async(self, actions):
        self.actions = actions
    
    def step_wait(self):
        # Simulate the environment one step forward in time with the given action
        self.step_counts += 1
        self.states = self.states + self.dt*f_func(self.states.T, self.actions.T).T

        # Rewards from https://github.com/uzh-rpg/flightmare/blob/competition/flightpy/configs/control/config.yaml
        pos_reward = -0.002*np.linalg.norm(self.states[:,0:3], axis=1)
        vel_reward = -0.002*np.linalg.norm(self.states[:,3:6], axis=1)
        ang_reward = -0.0001*np.linalg.norm(self.states[:,6:9], axis=1)
        rat_reward = -0.0001*np.linalg.norm(self.states[:,9:12], axis=1)

        rewards = pos_reward + vel_reward + ang_reward + rat_reward
        
        # Check if the goal has been reached
        pos_reached = np.linalg.norm(self.states[:,0:3], axis=1) < self.pos_threshold
        vel_reached = np.linalg.norm(self.states[:,3:6], axis=1) < self.vel_threshold
        ang_reached = np.all(np.abs(self.states[:,6:9]) < self.ang_threshold, axis=1)
        rat_reached = np.all(np.abs(self.states[:,9:12]) < self.rat_threshold, axis=1)

        goal_reached = pos_reached & vel_reached & ang_reached & rat_reached
        rewards[goal_reached] = 100

        # Check if out of bounds
        out_of_bounds = out_of_bounds = np.any(np.abs(self.states[:,0:3]) > 10, axis=1) | np.any(np.abs(self.states[:,6:8]) > np.pi, axis=1)
        rewards[out_of_bounds] = -1

        # Check number of steps
        max_steps_reached = self.step_counts >= self.max_steps

        # Check if the episode is done
        dones = goal_reached | out_of_bounds | max_steps_reached

        if np.any(max_steps_reached):
            print('max_steps_reached')
        if np.any(out_of_bounds):
            print('out_of_bounds')
        if np.any(goal_reached):
            print('goal_reached')

        # reset env if done
        self.reset_(dones)

        # Write info dicts
        infos = [{}] * self.num_envs
        for i in range(self.num_envs):
            if dones[i]:
                infos[i]["terminal_observation"] = self.states[i]
            if max_steps_reached[i] or out_of_bounds[i]:
                infos[i]["TimeLimit.truncated"] = True
        return self.states, rewards, dones, infos
    
    def close(self):
        pass

    def seed(self, seed=None):
        pass

    def get_attr(self, attr_name, indices=None):
        pass

    def set_attr(self, attr_name, value, indices=None):
        pass

    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        pass

    def env_is_wrapped(self, wrapper_class, indices=None):
        return [False]*self.num_envs

    def render(self, mode='human'):
        # Outputs a dict containing all information for rendering
        state_dict = dict(zip(['x','y','z','vx','vy','vz','phi','theta','psi','p','q','r','w1','w2','w3','w4'], self.states.T))
        # Rescale actions to [0,1] for rendering
        action_dict = dict(zip(['u1','u2','u3','u4'], (np.array(self.actions.T)+1)/2))
        return {**state_dict, **action_dict}

# Test animation code

In [ ]:
from quadcopter_animation import animation
# reload modules
import importlib
importlib.reload(animation)
import time

num = 100
env = Quadcopter3DVec(num_envs=num)

# Run a random agent
env.reset()
# print(env.render())
done = False
def run():
    global done
    action = np.random.uniform(-1,1, size=(num,4))
    state, reward, done, _ = env.step(action)
    # wait a bit
    # print(env.step_counts[0])
    return env.render()

animation.view(run)

# Define and Train the PPO agent

In [ ]:
import os
from stable_baselines3 import PPO
from datetime import datetime
from stable_baselines3.common.vec_env import VecMonitor

models_dir = 'models/ppo_3DquadVec'
log_dir = 'logs/ppo_3DquadVec'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Date and time string for unique folder names
datetime_str = datetime.now().strftime("%Y%m%d-%H%M%S")

# Create SubprocVecEnv for parallelization
# env = make_vec_env(lambda: Quadcopter3D(), n_envs=20)

# Create the environment
env = Quadcopter3DVec(num_envs=100)

# Wrap the environment in a Monitor wrapper
env = VecMonitor(env)

# ReLU net with 3 hidden layers of size 120
policy_kwargs = dict(
    activation_fn=torch.nn.ReLU,
    net_arch=[dict(pi=[120,120,120], vf=[120,120,120])],
    log_std_init = 0
)

model = PPO(
    "MlpPolicy",
    env,
    policy_kwargs=policy_kwargs,
    verbose=0,
    tensorboard_log=log_dir,
    n_steps=500,
    batch_size=5000,
    n_epochs=10,
)

print(model.device)

# add tanh layer to action net
from torch import nn
model.policy.action_net.add_module('tanh', nn.Tanh())
print(model.policy)

def train():
    TIMESTEPS = 10000
    for i in range(1,10000000000):
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name='PPO_'+datetime_str)
        model.save(models_dir + '/PPO_' + datetime_str + '/' + str(i))
        print('iteration: ', i, '  timestep: ', i*TIMESTEPS)

# train()

# Simulate performance

In [ ]:
from quadcopter_animation import animation
from stable_baselines3 import PPO
import os
import numpy as np
import importlib
importlib.reload(animation)

def animate_runs(model, env, **kwargs):
    env.reset()
    def run():
        actions, _ = model.predict(env.states)
        states, rewards, dones, infos = env.step(actions)
        return env.render()
    animation.view(run, **kwargs)

# make function that loads latest model from model folder
def load_latest_model(model_dir, index=-1):
    # get all folders in model_dir
    folders = os.listdir(model_dir)
    # get all folders that start with PPO_
    folders = [folder for folder in folders if folder.startswith('PPO_')]
    # get the latest folder
    latest_folder = sorted(folders)[index]
    print('latest_folder=',latest_folder)
    # get all files in the latest folder
    files = os.listdir(model_dir + '/' + latest_folder)
    # get the file with the highest number as name
    latest_file = max(files, key=lambda x: int(x[:-4]))
    print('latest_file=',latest_file)
    # load the latest model
    model = PPO.load(model_dir + '/' + latest_folder + '/' + latest_file)
    return model
    
models_dir = 'models/ppo_3DquadVec'
model = load_latest_model(models_dir)
animate_runs(model, Quadcopter3DVec(1))  

# Flying through gates!!

In [ ]:
# Efficient vectorized version of the environment
import torch
from gym import spaces
from stable_baselines3.common.vec_env import VecEnv

class Quadcopter3DVecGates(VecEnv):
    def __init__(self, num_envs, gates_pos, gate_yaw, start_pos):
        # action space and observation space (with extra dimension for the number of environments)
        action_space = spaces.Box(low=-1, high=1, shape=(4,))
        observation_space = spaces.Box(low=np.array([-np.inf]*12+[-1]*4), high=np.array([-np.inf]*12+[1]*4))
        VecEnv.__init__(self,num_envs, observation_space, action_space)

        # Define any other environment-specific parameters
        self.max_steps = 1000  # Maximum number of steps in an episode
        self.dt = 0.01  # Time step duration

        # Define the race track
        self.start_pos = start_pos.astype(np.float32)
        self.gate_pos = gates_pos.astype(np.float32)
        self.gate_yaw = gate_yaw.astype(np.float32)
        self.num_gates = gates_pos.shape[0]
        self.target_gates = np.zeros(num_envs, dtype=int)

        # Define any other necessary variables
        self.states = np.zeros((num_envs,16), dtype=np.float32)
        self.step_counts = np.zeros(num_envs, dtype=np.float32)
        self.actions = np.zeros((num_envs,4), dtype=np.float32)
        

    def reset_(self, dones):
        num_reset = dones.sum()

        # Sample random starting positions between the gates
        points = np.concatenate([self.start_pos[np.newaxis,:], self.gate_pos], axis=0)
        segment = np.random.randint(0, self.num_gates, size=(num_reset,))

        # Compute the starting position
        start_pos = (points[segment]+points[segment+1])/2
        # start_pos = self.start_pos[np.newaxis, :]

        x0 = 0.1*np.random.randn(num_reset) + start_pos[:,0]
        y0 = 0.1*np.random.randn(num_reset) + start_pos[:,1]
        z0 = 0.1*np.random.randn(num_reset) + start_pos[:,2]
        
        vx0 = 0.1*np.random.randn(num_reset)
        vy0 = 0.1*np.random.randn(num_reset)
        vz0 = 0.1*np.random.randn(num_reset)
        
        phi0   = 1*np.random.randn(num_reset)
        theta0 = 1*np.random.randn(num_reset)
        psi0   = 1*np.random.randn(num_reset)
        
        p0 = 0.1*np.random.randn(num_reset)
        q0 = 0.1*np.random.randn(num_reset)
        r0 = 0.1*np.random.randn(num_reset)
        
        w10 = np.random.uniform(-1,1, size=(num_reset,))
        w20 = np.random.uniform(-1,1, size=(num_reset,))
        w30 = np.random.uniform(-1,1, size=(num_reset,))
        w40 = np.random.uniform(-1,1, size=(num_reset,))

        self.states[dones] = np.stack([x0, y0, z0, vx0, vy0, vz0, phi0, theta0, psi0, p0, q0, r0, w10, w20, w30, w40], axis=1)
        self.step_counts[dones] = np.zeros(num_reset)
        self.target_gates[dones] = segment #np.zeros(num_reset, dtype=int)
        return self.states
    
    def reset(self):
        return self.reset_(np.ones(self.num_envs, dtype=bool))

    def step_async(self, actions):
        self.actions = actions
    
    def step_wait(self):
        # Simulate the environment one step forward in time with the given action
        self.step_counts += 1
        new_states = self.states + self.dt*f_func(self.states.T, self.actions.T).T

        pos_old = self.states[:,0:3]
        pos_new = new_states[:,0:3]
        pos_gate = self.gate_pos[self.target_gates]
        
        # Rewards from [secret paper]
        d2g_old = np.linalg.norm(pos_old - pos_gate, axis=1)
        d2g_new = np.linalg.norm(pos_new - pos_gate, axis=1)
        rat_penalty = 0.0001*np.linalg.norm(new_states[:,9:12], axis=1)
        rewards = d2g_old - d2g_new - rat_penalty
        
        # Check if the gate has been passed
        normal = np.array([np.cos(self.gate_yaw[self.target_gates]), np.sin(self.gate_yaw[self.target_gates])]).T
        # dot product of normal and position vector over axis 1
        pos_old_projected = (pos_old[:,0]-pos_gate[:,0])*normal[:,0] + (pos_old[:,1]-pos_gate[:,1])*normal[:,1]
        pos_new_projected = (pos_new[:,0]-pos_gate[:,0])*normal[:,0] + (pos_new[:,1]-pos_gate[:,1])*normal[:,1]
        passed_gate_plane = (pos_old_projected < 0) & (pos_new_projected > 0)
        gate_passed = passed_gate_plane & np.all(np.abs(pos_new - pos_gate)<0.5, axis=1)
        #rewards[gate_passed] = 10
        
        # Check for gate collision
        gate_collision = passed_gate_plane & np.any(np.abs(pos_new - pos_gate)>0.5, axis=1)
        rewards[gate_collision] = -10

        # Check ground collision (z > 0)
        ground_collision = self.states[:,2] > 0
        rewards[ground_collision] = -10
        
        # Check out of bounds
        # outside grid abs(x,y)>10
        # prevent numerical issues: abs(p,q,r) < 1000
        out_of_bounds = np.any(np.abs(self.states[:,0:2]) > 10, axis=1) | np.any(np.abs(self.states[:,9:12]) > 1000, axis=1)
        
        # Check number of steps
        max_steps_reached = self.step_counts >= self.max_steps

        # Update target gate
        self.target_gates[gate_passed] += 1
        
        # Check if final gate has been passed
        final_gate_passed = self.target_gates >= self.num_gates

        # give reward for passing final gate
        rewards[final_gate_passed] = 10
        
        # Check if the episode is done
        dones = max_steps_reached | gate_collision | ground_collision | final_gate_passed | out_of_bounds
        
        # Update states
        self.states = new_states
        
        # reset env if done
        self.reset_(dones)

        if np.any(gate_collision):
            print('gate_collision', gate_collision.sum())

        # Write info dicts
        infos = [{}] * self.num_envs
        for i in range(self.num_envs):
            if dones[i]:
                infos[i]["terminal_observation"] = self.states[i]
            if max_steps_reached[i]:
                infos[i]["TimeLimit.truncated"] = True
        return self.states, rewards, dones, infos
    
    def close(self):
        pass

    def seed(self, seed=None):
        pass

    def get_attr(self, attr_name, indices=None):
        pass

    def set_attr(self, attr_name, value, indices=None):
        pass

    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        pass

    def env_is_wrapped(self, wrapper_class, indices=None):
        return [False]*self.num_envs

    def render(self, mode='human'):
        # Outputs a dict containing all information for rendering
        state_dict = dict(zip(['x','y','z','vx','vy','vz','phi','theta','psi','p','q','r','w1','w2','w3','w4'], self.states.T))
        # Rescale actions to [0,1] for rendering
        action_dict = dict(zip(['u1','u2','u3','u4'], (np.array(self.actions.T)+1)/2))
        return {**state_dict, **action_dict}

# Define Race Track

In [ ]:
importlib.reload(animation)

# Define the race track
gate_pos = np.array([
    [-1.5,-2,-1.5],
    [1.5,2,-1.5],
    [1.5,-2,-1.5],
    [-1.5,2,-1.5]
]*2)

gate_yaw = np.array([
    0,
    0,
    np.pi,
    np.pi,
]*2)

start_pos = np.array([-4,-2,-1.5])

# gate_pos = gate_pos[0:2]
# gate_yaw = gate_yaw[0:2]

# Test Animation Code

In [ ]:
from quadcopter_animation import animation
# reload modules
import importlib
importlib.reload(animation)
import time

num = 10
env = Quadcopter3DVecGates(num_envs=num, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos)

# Run a random agent
env.reset()
# print(env.render())
done = False
def run():
    global done
    action = np.random.uniform(-1,1, size=(num,4))
    state, reward, done, _ = env.step(action)
    # wait a bit
    # print(env.step_counts[0])
    return env.render()

animation.view(run, gate_pos=gate_pos, gate_yaw=gate_yaw)

In [ ]:
import os
from stable_baselines3 import PPO
from datetime import datetime
from stable_baselines3.common.vec_env import VecMonitor

models_dir = 'models/ppo_3DquadVecGates'
log_dir = 'logs/ppo_3DquadVecGates'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Date and time string for unique folder names
datetime_str = datetime.now().strftime("%Y%m%d-%H%M%S")

# Create the environment
env = Quadcopter3DVecGates(num_envs=100, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos)

# Wrap the environment in a Monitor wrapper
env = VecMonitor(env)

# ReLU net with 3 hidden layers of size 120
policy_kwargs = dict(
    activation_fn=torch.nn.ReLU,
    net_arch=[dict(pi=[120,120,120], vf=[120,120,120])],
    log_std_init = 0
)

model = PPO(
    "MlpPolicy",
    env,
    policy_kwargs=policy_kwargs,
    verbose=0,
    tensorboard_log=log_dir,
    n_steps=1000,
    batch_size=5000,
    n_epochs=10,
)

print(model.device)

# add tanh layer to action net
from torch import nn
model.policy.action_net.add_module('tanh', nn.Tanh())
print(model.policy)

def train(model, train_env, test_env):
    TIMESTEPS = model.n_steps*env.num_envs*10
    for i in range(1,10000000000):
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name='PPO_'+datetime_str)
        model.save(models_dir + '/PPO_' + datetime_str + '/' + str(i))
        # animate_progress(model, test_env)

def animate_progress(model, env):
    env.reset()
    def run():
        actions, _ = model.predict(env.states)
        states, rewards, dones, infos = env.step(actions)
        return env.render()
    animation.view(run, gate_pos=gate_pos, gate_yaw=gate_yaw)

# run training loop
test_env = Quadcopter3DVecGates(num_envs=10, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos)
train(model, env, test_env)

In [ ]:
import importlib
importlib.reload(animation)

def animate_progress(models_dir, env, **kwargs):
    model = load_latest_model(models_dir, index=-1)
    env.reset()
    def run():
        actions, _ = model.predict(env.states)
        states, rewards, dones, infos = env.step(actions)
        # print('rewards', rewards)
        return env.render()
    animation.view(run, **kwargs)

test_env = Quadcopter3DVecGates(num_envs=1, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos)
models_dir = 'models/ppo_3DquadVecGates'
animate_progress(models_dir, test_env, gate_pos=gate_pos, gate_yaw=gate_yaw)  

In [ ]:
# kill training thread
5.77/2